<a href="https://colab.research.google.com/github/MuhammedShiway/Differential-Convolution-Modifications/blob/main/%5BCompleted%5D_Manually_Testing_DiffConv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import torch.nn as nn
from torchvision import transforms
from PIL import Image

import torch
import numpy as np
import torch.nn as nn

#EXTREMES NORMALIZER
def normalize_ext(x):
  high = torch.max(x)
  low = torch.min(x)
  x = ((x-low) * high) / (high - low);
  return x

#NORMALIZER USING ONLY MAX
def normalize_max(x):
  high = torch.max(x)
  low = 0 - high
  x = ((x-low) * high) / (high - low);
  return x

#NORMALIZER USING DATA MAX
def normalize_data_max(x, data):
  high = torch.max(data)
  low = 0 - high
  x = ((x-low) * high) / (high - low);
  return x

#SIGMOID NORMALIZER
def normalize_sig(x):
  x = torch.sigmoid(x)
  return x

#MAX_SIGMOID NORMALIZER
def normalize_sig_max(x):
  high = torch.max(x)
  x = high / (1+ torch.exp(-x))
  return x

class DiffConv(nn.Module):

    def __init__(self):
        super(DiffConv, self).__init__()




    def forward(self, x, normalize):
        sizeofin = x.size()

        ins = sizeofin[0]
        n = sizeofin[1]
        sx = sizeofin[2]
        sy = sizeofin[3]

        self.output = torch.zeros(ins,n*5,sx,sy, dtype=torch.int32).cuda()
        self.signInputs = torch.zeros(ins,n*5,sx,sy).cuda()

        x = x.cuda()
        for i in range(0,ins):


            oM = 0
            oN = n

            self.output[i,oM:oN,0:sx,0:sy]= x[i].clone().cuda()

            oM = n
            oN = 2*n

            #area = self.output[i,oM:oN,0:sx-1,0:sy]
            self.output[i,oM:oN,0:sx-1,0:sy] = self.output[i,oM:oN,0:sx-1,0:sy].add(x[i,0:n,0:sx-1,0:sy])
            self.output[i,oM:oN,0:sx-1,0:sy] = self.output[i,oM:oN,0:sx-1,0:sy].add(-x[i,0:n,1:sx,0:sy])

            oM = 2*n
            oN = 3*n

            #area = ptr[oM:oN,0:sx,0:sy-1]
            self.output[i,oM:oN,0:sx,0:sy-1]=self.output[i,oM:oN,0:sx,0:sy-1].add(x[i,0:n,0:sx,0:sy-1])
            self.output[i,oM:oN,0:sx,0:sy-1]=self.output[i,oM:oN,0:sx,0:sy-1].add(-x[i,0:n,0:sx,1:sy])

            oM = 3*n
            oN = 4*n

            #area = ptr[oM:oN,0:sx-1,0:sy-1]
            self.output[i,oM:oN,0:sx-1,0:sy-1]=self.output[i,oM:oN,0:sx-1,0:sy-1].add(x[i,0:n,0:sx-1,0:sy-1])
            self.output[i,oM:oN,0:sx-1,0:sy-1]=self.output[i,oM:oN,0:sx-1,0:sy-1].add(-x[i,0:n,1:sx,1:sy])

            oM = 4*n
            oN = 5*n

            #area = ptr[oM:oN,0:sx-1,0:sy-1]
            self.output[i,oM:oN,0:sx-1,0:sy-1]=self.output[i,oM:oN,0:sx-1,0:sy-1].add(x[i,0:n,1:sx,0:sy-1])
            self.output[i,oM:oN,0:sx-1,0:sy-1]=self.output[i,oM:oN,0:sx-1,0:sy-1].add(-x[i,0:n,0:sx-1,1:sy])


        oM = n
        oN = 5*n


        #self.signInputs = self.output.sign()
        #self.signInputs[0:ins,0:n,:,:] = torch.ones(ins,n,sx,sy)

        #Inplace error came from the line below
        #self.output[0:ins,oM:oN,:,:] = self.output[0:ins,oM:oN].abs()

        #No more in place error


        if normalize == 'abs':
          self.output[0:ins,oM:oN,:,:] = torch.abs(self.output[0:ins,oM:oN].clone().detach());
        elif normalize == 'ext':
          self.output[0:ins,oM:oN,:,:] = normalize_ext(self.output[0:ins,oM:oN].clone().detach() );
        elif normalize == 'max':
          self.output[0:ins,oM:oN,:,:] = normalize_max(self.output[0:ins,oM:oN].clone().detach());
        elif normalize == 'data_max':
          self.output[0:ins,oM:oN,:,:] = normalize_data_max(self.output[0:ins,oM:oN].clone().detach(), data = x);
        elif normalize == 'sig':
          self.output[0:ins,oM:oN,:,:] = normalize_sig(self.output[0:ins,oM:oN].clone().detach());
        elif normalize == 'sig_max':
          self.output[0:ins,oM:oN,:,:] = normalize_sig_max(self.output[0:ins,oM:oN].clone().detach());
        elif normalize == 'raw':
          self.output[0:ins,oM:oN,:,:] = self.output[0:ins,oM:oN].clone().detach();






        #print(self.output)

        return self.output



In [ ]:
a = torch.rand(10,10)

a = torch.abs(a)

In [ ]:
a

tensor([[[[0.6741, 0.2914, 0.5378, 0.0971, 0.3715, 0.5030, 0.1403, 0.0639,
           0.0914, 0.4732],
          [0.9690, 0.2094, 0.1765, 0.5146, 0.3357, 0.7768, 0.4023, 0.3151,
           0.8894, 0.4172],
          [0.8109, 0.8006, 0.9561, 0.6758, 0.8673, 0.3738, 0.0716, 0.7503,
           0.9448, 0.7208],
          [0.7972, 0.9362, 0.6674, 0.6604, 0.3199, 0.4121, 0.0186, 0.1636,
           0.6076, 0.0758],
          [0.1514, 0.6202, 0.8776, 0.5671, 0.8455, 0.5851, 0.7834, 0.9009,
           0.8829, 0.5266],
          [0.2263, 0.8124, 0.6994, 0.4495, 0.1819, 0.7057, 0.3042, 0.8938,
           0.3351, 0.6176],
          [0.0951, 0.6675, 0.7012, 0.9109, 0.9807, 0.7835, 0.2794, 0.2344,
           0.4533, 0.3897],
          [0.1949, 0.1210, 0.7520, 0.4888, 0.6349, 0.9109, 0.1117, 0.5958,
           0.2970, 0.2745],
          [0.1846, 0.2423, 0.2745, 0.2573, 0.1801, 0.5112, 0.8111, 0.3419,
           0.3685, 0.6556],
          [0.6810, 0.6581, 0.2933, 0.9527, 0.3977, 0.8861, 0.0754, 0.3624

In [ ]:
#NOW WE APPLY DIFFCONV AND SEE THE RESULT
p = DiffConv()

In [ ]:
t = p.forward(a, normalize = 'abs')
t1 = p.forward(a, normalize = 'ext')
t2 = p.forward(a, normalize = 'raw')

In [ ]:
t

tensor([[[[0.6741, 0.2914, 0.5378, 0.0971, 0.3715, 0.5030, 0.1403, 0.0639,
           0.0914, 0.4732],
          [0.9690, 0.2094, 0.1765, 0.5146, 0.3357, 0.7768, 0.4023, 0.3151,
           0.8894, 0.4172],
          [0.8109, 0.8006, 0.9561, 0.6758, 0.8673, 0.3738, 0.0716, 0.7503,
           0.9448, 0.7208],
          [0.7972, 0.9362, 0.6674, 0.6604, 0.3199, 0.4121, 0.0186, 0.1636,
           0.6076, 0.0758],
          [0.1514, 0.6202, 0.8776, 0.5671, 0.8455, 0.5851, 0.7834, 0.9009,
           0.8829, 0.5266],
          [0.2263, 0.8124, 0.6994, 0.4495, 0.1819, 0.7057, 0.3042, 0.8938,
           0.3351, 0.6176],
          [0.0951, 0.6675, 0.7012, 0.9109, 0.9807, 0.7835, 0.2794, 0.2344,
           0.4533, 0.3897],
          [0.1949, 0.1210, 0.7520, 0.4888, 0.6349, 0.9109, 0.1117, 0.5958,
           0.2970, 0.2745],
          [0.1846, 0.2423, 0.2745, 0.2573, 0.1801, 0.5112, 0.8111, 0.3419,
           0.3685, 0.6556],
          [0.6810, 0.6581, 0.2933, 0.9527, 0.3977, 0.8861, 0.0754, 0.3624

In [ ]:
t1

tensor([[[[0.6741, 0.2914, 0.5378, 0.0971, 0.3715, 0.5030, 0.1403, 0.0639,
           0.0914, 0.4732],
          [0.9690, 0.2094, 0.1765, 0.5146, 0.3357, 0.7768, 0.4023, 0.3151,
           0.8894, 0.4172],
          [0.8109, 0.8006, 0.9561, 0.6758, 0.8673, 0.3738, 0.0716, 0.7503,
           0.9448, 0.7208],
          [0.7972, 0.9362, 0.6674, 0.6604, 0.3199, 0.4121, 0.0186, 0.1636,
           0.6076, 0.0758],
          [0.1514, 0.6202, 0.8776, 0.5671, 0.8455, 0.5851, 0.7834, 0.9009,
           0.8829, 0.5266],
          [0.2263, 0.8124, 0.6994, 0.4495, 0.1819, 0.7057, 0.3042, 0.8938,
           0.3351, 0.6176],
          [0.0951, 0.6675, 0.7012, 0.9109, 0.9807, 0.7835, 0.2794, 0.2344,
           0.4533, 0.3897],
          [0.1949, 0.1210, 0.7520, 0.4888, 0.6349, 0.9109, 0.1117, 0.5958,
           0.2970, 0.2745],
          [0.1846, 0.2423, 0.2745, 0.2573, 0.1801, 0.5112, 0.8111, 0.3419,
           0.3685, 0.6556],
          [0.6810, 0.6581, 0.2933, 0.9527, 0.3977, 0.8861, 0.0754, 0.3624

In [ ]:
t2

tensor([[[[ 0.6741,  0.2914,  0.5378,  0.0971,  0.3715,  0.5030,  0.1403,
            0.0639,  0.0914,  0.4732],
          [ 0.9690,  0.2094,  0.1765,  0.5146,  0.3357,  0.7768,  0.4023,
            0.3151,  0.8894,  0.4172],
          [ 0.8109,  0.8006,  0.9561,  0.6758,  0.8673,  0.3738,  0.0716,
            0.7503,  0.9448,  0.7208],
          [ 0.7972,  0.9362,  0.6674,  0.6604,  0.3199,  0.4121,  0.0186,
            0.1636,  0.6076,  0.0758],
          [ 0.1514,  0.6202,  0.8776,  0.5671,  0.8455,  0.5851,  0.7834,
            0.9009,  0.8829,  0.5266],
          [ 0.2263,  0.8124,  0.6994,  0.4495,  0.1819,  0.7057,  0.3042,
            0.8938,  0.3351,  0.6176],
          [ 0.0951,  0.6675,  0.7012,  0.9109,  0.9807,  0.7835,  0.2794,
            0.2344,  0.4533,  0.3897],
          [ 0.1949,  0.1210,  0.7520,  0.4888,  0.6349,  0.9109,  0.1117,
            0.5958,  0.2970,  0.2745],
          [ 0.1846,  0.2423,  0.2745,  0.2573,  0.1801,  0.5112,  0.8111,
            0.3419,  0